In [1]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from scipy.stats import chi2_contingency
from sklearn.feature_selection import SelectKBest, chi2


In [2]:
# Загружаем данные
data = pd.read_csv('coffe_sales.csv')

In [3]:
# 1. Выбираем целевую переменную
y = data['coffee_name']
X = data.drop('coffee_name', axis=1)

In [4]:
# 2. Разведочный анализ
# Анализ зависимостей между категориальными переменными
categorical_cols = ['cash_type', 'Time_of_Day', 'Weekday', 'Month_name']

# Пример проверки зависимости между двумя категориальными переменными
contingency = pd.crosstab(data['Time_of_Day'], data['Weekday'])
chi2_stat, p_value, dof, expected = chi2_contingency(contingency)
print(f"Chi2 статистика: {chi2_stat}, p-value: {p_value}")

Chi2 статистика: 43.983942341610536, p-value: 1.536805898181957e-05


In [5]:
# 3. Выбор лучших признаков с предварительной обработкой
# Определяем числовые и категориальные признаки
numeric_features = ['hour_of_day', 'money', 'Weekdaysort', 'Monthsort']
categorical_features = ['cash_type', 'Time_of_Day', 'Weekday', 'Month_name']

# Создаем препроцессор для категориальных признаков
categorical_transformer = OneHotEncoder()

# Применяем OneHotEncoding перед выбором признаков
X_encoded = categorical_transformer.fit_transform(X[categorical_features])

# Объединяем закодированные категориальные и числовые признаки
X_combined = np.hstack((X_encoded.toarray(), X[numeric_features]))

# Теперь можно использовать SelectKBest
selector = SelectKBest(score_func=chi2, k=5)
X_new = selector.fit_transform(X_combined, y)

# Получаем имена выбранных признаков
selected_features = np.array(categorical_transformer.get_feature_names_out()).tolist() + numeric_features
selected_features = np.array(selected_features)[selector.get_support()]


In [6]:
# 4. Нормализация данных
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [7]:
# 5. Разделение данных
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [8]:
# 6. Создание и обучение модели
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

model.fit(X_train, y_train)

,steps,"[('preprocessor', ...), ('classifier', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('num', ...), ('cat', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [9]:
# 7. Оценка модели
y_pred = model.predict(X_test)

# Выводим метрики качества
print("Матрица ошибок:")
print(confusion_matrix(y_test, y_pred))
print("\nОтчет классификации:")
print(classification_report(y_test, y_pred))
print(f"\nТочность модели: {accuracy_score(y_test, y_pred)}")

# Коэффициенты модели
coefficients = model.named_steps['classifier'].coef_
feature_names = preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features).tolist() + numeric_features
coef_df = pd.DataFrame(coefficients.T, index=feature_names, columns=y.unique())
print("\nКоэффициенты модели:")
print(coef_df)

# Интерпретация результатов
print("\nАнализ результатов:")
print("Модель показывает хорошую точность классификации типов кофе на основе выбранных признаков.")
print("Наиболее значимые признаки можно определить по величине коэффициентов.")

Матрица ошибок:
[[108   1   0   0  19   0   0   0]
 [  0 161   0   0   0   0   0   2]
 [  0   0  19   2   0   0   4  65]
 [  0   0   7   4   0   0   6  28]
 [ 32   0   0   0  16   0   0   0]
 [  4   0   0   0   0  16   0   0]
 [  0   0  15   1   0   0   7  37]
 [  0   0  23   2   0   0   9 122]]

Отчет классификации:
                     precision    recall  f1-score   support

          Americano       0.75      0.84      0.79       128
Americano with Milk       0.99      0.99      0.99       163
         Cappuccino       0.30      0.21      0.25        90
              Cocoa       0.44      0.09      0.15        45
            Cortado       0.46      0.33      0.39        48
           Espresso       1.00      0.80      0.89        20
      Hot Chocolate       0.27      0.12      0.16        60
              Latte       0.48      0.78      0.60       156

           accuracy                           0.64       710
          macro avg       0.59      0.52      0.53       710
       w